In [1]:
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import requests
import time
import datetime
import pymysql

In [104]:
api_key = 'RGAPI-bee4c76f-7883-43ca-a8ae-45131dbd1cc1'
api = '?api_key='+api_key

# 1. 티어별 소환사명 목록 만들기

## Challenger to Master

In [4]:
ch_queue_req = requests.get('https://kr.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'+api)
gm_queue_req = requests.get('https://kr.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5'+api)
ms_queue_req = requests.get('https://kr.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5'+api)

In [26]:
challengers = json_normalize(ch_queue_req.json()['entries'])
grandmasters = json_normalize(gm_queue_req.json()['entries'])
masters = json_normalize(ms_queue_req.json()['entries'])

ch_to_ms = pd.concat([challengers, grandmasters, masters], ignore_index=True)

print('# of summoners:    ', len(ch_to_ms))
print('# of challengers:  ', len(challengers))
print('# of grandmasters: ', len(grandmasters))
print('# of masters:      ', len(masters))
print()

ch_to_ms.tail()

# of summoners:     2786
# of challengers:   300
# of grandmasters:  700
# of masters:       1786



,freshBlood,hotStreak,inactive,leaguePoints,losses,rank,summonerId,summonerName,veteran,wins
2781,True,False,False,0,158,I,0Hn2MkvroCiiRPljQ6hckWC3QF7VKiptD_q86qM4i67yCVE,재 준,False,177
2782,False,False,False,50,56,I,DDvzdLYWNH4ck843tPigx7uqChjDISj34b1VnnQsGvE99g,We알the월드,False,79
2783,True,True,False,0,213,I,JkAnII8p9jIqOy-1oMGLcVgyC5GP5uNZEnSJf-joCIj2ghY,손세이,False,257
2784,True,True,False,320,302,I,Sc3pTbVWi0uU8TdJoFbf5p2b1Nmjkgos8NrQLNizLQpIekU,벙댕이벙댕이,False,328
2785,False,False,False,150,399,I,a2WLwPOfhSlco0x8_37svHSckHROi65YtMqyzWV7GoREQW4,pokerfacezz,False,443


In [95]:
ch_to_ms.to_csv('./data/ch_to_ms.csv')

## Challenger to Diamond2

In [ ]:
ch_queue_req = requests.get('https://kr.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'+api)
gm_queue_req = requests.get('https://kr.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5'+api)
ms_queue_req = requests.get('https://kr.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5'+api)
dia1_queue_req = requests.get('https://kr.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/DIAMOND/I?page=50&api_key='+api_key)
dia2_queue_req = requests.get('https://kr.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/DIAMOND/II?page=50&api_key='+api_key)
dia3_queue_req = requests.get('https://kr.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/DIAMOND/III'+api)
dia4_queue_req = requests.get('https://kr.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/DIAMOND/IV?page=100&api_key='+api_key)

In [ ]:
challengers = json_normalize(ch_queue_req.json()['entries'])
grandmasters = json_normalize(gm_queue_req.json()['entries'])
masters = json_normalize(ms_queue_req.json()['entries'])
dias = {}

ch_to_dia = pd.concat([challengers, grandmasters, masters], ignore_index=True)

for tier in ['I', 'II']:
    print('==================', tier, '==================')
    dias[tier] = pd.DataFrame()
    
    page = 1
    
    while(True):
#         print(page)
        dia_queue_req = requests.get('https://kr.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/DIAMOND/'+tier +
                                     '?page=' + str(page) + '&api_key='+api_key)
        
        if dia_queue_req.status_code == 429:
            time.sleep(120)
            continue
            
        if len(dia_queue_req.content) == 2:
            print(tier, page)
            break
        else:
            temp_dias = json_normalize(dia_queue_req.json())
            dias[tier] = dias[tier].append(temp_dias)
            page += 1
       
    ch_to_dia = pd.concat([ch_to_dia, dias[tier]], ignore_index=True)

print('# of summoners:    ', len(ch_to_dia))
print('# of challengers:  ', len(challengers))
print('# of grandmasters: ', len(grandmasters))
print('# of masters:      ', len(masters))
print('# of diamond1:     ', len(dias['I']))
print('# of diamond2:     ', len(dias['II']))
print()

ch_to_dia.tail()

In [ ]:
column_order = ['summonerName', 'summonerId', 'tier', 'rank', 'wins', 'losses',
                            'leagueId', 'leaguePoints', 'queueType', 'veteran', 
                            'freshBlood', 'hotStreak', 'inactive', 
                             'miniSeries.wins', 'miniSeries.losses', 'miniSeries.progress', 'miniSeries.target']

ch_to_dia_df = ch_to_dia_df[column_order]
ch_to_dia_df.to_csv('./data/ch_to_dia.csv', columns=column_order)

# 2. 소환사명으로 소환사 데이터 불러오기

In [ ]:
summoners_df = pd.DataFrame()

i = 0
N = len(ch_to_ms.summonerName.values)

In [ ]:
while(i < N):
    
    sn = ch_to_ms.summonerName.values[i]
    
    try:
        summoner_req = requests.get('https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/'+sn+api)
        summoner_df = json_normalize(summoner_req.json())
        print('{0}   {1:5d}: {2}'.format(datetime.datetime.now().strftime("%H:%M:%S"), i, summoner_df.name[0]))
    except Exception as e:
        if summoner_df['status.status_code'][0] == 404:
            print('{0}   {1} '.format(datetime.datetime.now().strftime("%H:%M:%S"), e))
            pass
        else:
            time.sleep(125)
            continue
        
    summoners_df = summoners_df.append(summoner_df, ignore_index=True)
    i += 1
    
    if (i%100) == 9:
        time.sleep(125)

In [25]:
summoners_df.head()

,accountId,id,name,profileIconId,puuid,revisionDate,status.message,status.status_code,summonerLevel
0,Ejv_VgDh-CqOFPs7fQ3O09t0piOlJ04cWj7Kd5eVfg7mNeg,4KZNxdRDCuYdJ4Bk-FB7-C5wFNOlZRc6iDWfLNlzPJr_p3g,Koei,4657.0,ziRbKduN9LL-8Zvuh8SknDa91kBAgGrU-zCBsWckUJtssA...,1.596096e+12,NaN,NaN,389.0
1,SBjNgdkpuz94vJBC9yHmgxX68lRSWvEev9EeIkeT6vFV,JJv1hB9eCeJBYuUjibIduPoMkTT3WeHCH3F9IDVRCdNEJg,SHARK KING,4026.0,1yNhmcdjxj7N2nhjr4Ym1bAPxqLK74Lr6psmiKSW0f8UCx...,1.596069e+12,NaN,NaN,417.0
2,CbHFTiaK0gkgN7EEf6xBLKZ2A-W_uRHpze_EvA1eh_Cq9m0,hchRe823zSgHMfH98Yw6jf5LrYeaWQKkq6VZftclRdx5ldo,tian xia wu di,3868.0,CWNHIC1XyHjqQLrIfwjW-oYtkYRNFB_8pzi_E-hsrgEAmV...,1.596064e+12,NaN,NaN,399.0
3,7iNHYnfzhFGQDpgjE-cLPEisWEctzD7zsAndGQDWRvReuH...,ej4nswcyRKLEvOlblgyscFkWiDIGffTYS8kblr317WmSt04,qqiqi77,3838.0,17l-PaeGYwkUE6vejt5gXI-nH65x2NU-u6GI07vwACKIQ2...,1.596019e+12,NaN,NaN,346.0
4,uLQMOpLfS70FscLFQ7BwuvZwvWgnq0Ba8C-jMnzBHWYcUT...,8Qda48p8GR1dB9tN2U2UrgDUIkra7X0FFLCoMlIE-U1ief4,도넛이맛어요,4637.0,zUafDfs6FS_ZVpW0SnXjDkpA6H7P642VcFtKhA0vEGlCRn...,1.596015e+12,NaN,NaN,157.0


In [24]:
summoners_df.to_csv('./data/summoners_df.csv')

In [34]:
# 합치기
# summoners_df.drop(columns=['id'], inplace=True)
summoners_df.rename(columns={'name':'summonerName'}, inplace=True)

SUMMONERS = pd.merge(ch_to_ms, summoners_df, on='summonerName', how='inner')
SUMMONERS.drop(columns=['status.message', 'status.status_code'], inplace=True)
SUMMONERS.to_csv('./data/ch_to_ms.csv')
SUMMONERS.tail()

,freshBlood,hotStreak,inactive,leaguePoints,losses,rank,summonerId,summonerName,veteran,wins,accountId,id,profileIconId,puuid,revisionDate,summonerLevel
2758,True,False,False,0,158,I,0Hn2MkvroCiiRPljQ6hckWC3QF7VKiptD_q86qM4i67yCVE,재 준,False,177,z5oE2ANOCljLxjqYOmrhP8HRcmXDL8cnHVjxI3_vE1PVlv...,0Hn2MkvroCiiRPljQ6hckWC3QF7VKiptD_q86qM4i67yCVE,4297.0,uXsaVmiC0sym02RhEw-hnLsaQLc3yHCsAYADGarFRTNk6v...,1.596132e+12,209.0
2759,False,False,False,50,56,I,DDvzdLYWNH4ck843tPigx7uqChjDISj34b1VnnQsGvE99g,We알the월드,False,79,nkA6WzvXMxJG6yCm2tDhEYaP5qN5lAy-Om0TJKXUlGI6,DDvzdLYWNH4ck843tPigx7uqChjDISj34b1VnnQsGvE99g,4162.0,q1Ba76ALoZTPzgxcpXTMQL0wR568a5rJrxVDvwk8krM4IJ...,1.595678e+12,91.0
2760,True,True,False,0,213,I,JkAnII8p9jIqOy-1oMGLcVgyC5GP5uNZEnSJf-joCIj2ghY,손세이,False,257,Joxm2i7Cv8FzcoMsN2LsEMt34lMZnPjE78ByJ7JcLZCNVxk,JkAnII8p9jIqOy-1oMGLcVgyC5GP5uNZEnSJf-joCIj2ghY,4561.0,Rvc6ow901DqzZS2thEBeeuo1pOacPUhBzQgz_vRKnddwaz...,1.596102e+12,208.0
2761,True,True,False,320,302,I,Sc3pTbVWi0uU8TdJoFbf5p2b1Nmjkgos8NrQLNizLQpIekU,벙댕이벙댕이,False,328,Cisx5vOO_zrUAyyCoWMzEjqnn08SqssUuNrk-ZyNChgfMR8,Sc3pTbVWi0uU8TdJoFbf5p2b1Nmjkgos8NrQLNizLQpIekU,983.0,tyWXNL4Blwt_8iEGd1MIRfcrzvIaH0a9hgaJdm78umiBiv...,1.596052e+12,243.0
2762,False,False,False,150,399,I,a2WLwPOfhSlco0x8_37svHSckHROi65YtMqyzWV7GoREQW4,pokerfacezz,False,443,d6bPWaj4NVr_6bVXP_vlF35FScivPM0pDbE-vHICImI_L2...,a2WLwPOfhSlco0x8_37svHSckHROi65YtMqyzWV7GoREQW4,4568.0,_1OlL3KsJsC_ndqYzovOTtRIvRKtTxmlSvH2Sr6Yj3sM1N...,1.596078e+12,123.0


# 3. 소환사 계정 ID로 match 데이터 불러오기

In [ ]:
SUMMONERS = pd.read_csv('./data/ch_to_ms.csv')

In [ ]:
# matches_set = set()

N = len(SUMMONERS)
i = 1500

In [ ]:
while(i < N):
    
    ai = SUMMONERS.accountId.values[i]
    
    try:
        matches_req = requests.get('https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/'+ai+api)
        matches_df = json_normalize(matches_req.json()['matches'])
        print('{0}   {1:5d}: {2:5d}'.format(datetime.datetime.now().strftime("%H:%M:%S"), i, len(matches_set)))
        
    except Exception as e:
        #if matches_req.status_code == 403 or matches_req.status_code == 404:
        if matches_req.status_code == 404:
            print('{0}   {1} '.format(datetime.datetime.now().strftime("%H:%M:%S"), matches_req.status_code))
            pass
        else:
            print('{0}   {1} '.format(datetime.datetime.now().strftime("%H:%M:%S"), e))
            time.sleep(5)
            continue
        
    matches_set.update(matches_df.gameId.values)
    i += 1
    
    if (i%100) == 0:
        time.sleep(125)

In [57]:
with open('./data/matches_set.txt', 'w') as file:
    for match in list(matches_set):
        file.write(str(match)+'\n')

# 4. match id로 match 데이터 불러오기

In [8]:
matches_df = pd.read_csv('./data/matches_set.txt', header=None, names=['match_id'])
matches_list = matches_df['match_id'].values
print(len(matches_list))

124849


In [112]:
# MATCHES = pd.DataFrame()

N = len(matches_list)
j = 39171

In [ ]:
while(j < N):
    
    mi = str(matches_list[j])
    
    try:
        match_req = requests.get('https://kr.api.riotgames.com/lol/match/v4/matches/'+mi+api)
        match_df = json_normalize(match_req.json())
        gameId = match_df.gameId[0]
#         print('{0}   {1:5d}: {2}'.format(datetime.datetime.now().strftime("%H:%M:%S"), j, match_df.gameId[0]))
    except Exception as e:
        if match_req.status_code == 405 or match_req.status_code == 403:
            print('{0}   {1} '.format(datetime.datetime.now().strftime("%H:%M:%S"), match_req.status_code))
            pass
        else:
            print('{0}   {1} '.format(datetime.datetime.now().strftime("%H:%M:%S"), e))
            time.sleep(5)
            continue
    
    try:
        #=============================================================
        #match_df 정리
        #=============================================================
        #match_df.drop(columns=['gameCreation', 'gameMode', 'gameType',
        #                   'mapId', 'queueId'], inplace=True)

        prtcp_key_list = ['championId', 'spell1Id', 'spell2Id']
        prtcpId_key_list = ['accountId', 'summonerName', 'summonerId']


        #승리 팀--------------------------------------
        team1 = match_df['teams'].values[0][0]['win']
        team2 = match_df['teams'].values[0][1]['win']

        if team1 == 'Win':
            match_df['win'] = 'team1'
        else:
            match_df['win'] = 'team2'
        #---------------------------------------------


        #lane 결정-------------------------------------------
        def select_lane(lane, role):
            if lane == 'TOP':
                return 'TOP'
            elif lane == 'JUNGLE':
                return 'JUG'
            elif lane == 'MIDDLE':
                return 'MID'
            elif lane == 'BOTTOM' and role == 'DUO_CARRY':
                return 'ADC'
            elif lane == 'BOTTOM' and role == 'DUO_SUPPORT':
                return 'SUP'
            else:
                return None
        #---------------------------------------------------


        #team 1 데이터 정리----------------------------------------------------
        for i in range(5):
    #         lane = match_df['participants'].values[0][i]['timeline']['lane']
    #         role = match_df['participants'].values[0][i]['timeline']['role']

    #         lane = select_lane(lane, role)
    #         if lane == None:
    #             lane = str(i)

            for key in prtcp_key_list:
                match_df['team1_{0}_{1}'.format(i+1, key)] = match_df['participants'].values[0][i][key]

            for key in prtcpId_key_list:
                match_df['team1_{0}_{1}'.format(i+1, key)] = match_df['participantIdentities'].values[0][i]['player'][key]

            for key in ['lane', 'role']:
                match_df['team1_{0}_{1}'.format(i+1, key)] = match_df['participants'].values[0][i]['timeline'][key]
        #----------------------------------------------------------------------

        #team 2 데이터 정리-----------------------------------------------------
        for i in range(5,10):
    #         lane = match_df['participants'].values[0][i]['timeline']['lane']
    #         role = match_df['participants'].values[0][i]['timeline']['role']

            lane = select_lane(lane, role)

            for key in prtcp_key_list:
                match_df['team2_{0}_{1}'.format(i-4, key)] = match_df['participants'].values[0][i][key]

            for key in prtcpId_key_list:
                match_df['team2_{0}_{1}'.format(i-4, key)] = match_df['participantIdentities'].values[0][i]['player'][key]

            for key in ['lane', 'role']:
                match_df['team2_{0}_{1}'.format(i-4, key)] = match_df['participants'].values[0][i]['timeline'][key]
        #-----------------------------------------------------------------------
        
        match_df.drop(columns=['teams', 'participants', 'participantIdentities'], inplace=True)
        MATCHES = MATCHES.append(match_df, ignore_index=True)
        
    except Exception as e:
        print('{0}   {1:5d}: {2} {3}'.format(datetime.datetime.now().strftime("%H:%M:%S"), j, match_df.gameId[0], e))
        
    j += 1
    
    if (j%100) == 0:
        print('{0}   {1:5d}: {2}'.format(datetime.datetime.now().strftime("%H:%M:%S"), j, gameId))
        MATCHES.to_csv('./data/matches.csv')
        time.sleep(120)

04:23:08   39173: 4498473434 'win'
04:23:39   39200: 4516823555
04:26:59   39269: 4508959423 'summonerId'
04:27:36   39300: 4458103544
04:30:27   'DataFrame' object has no attribute 'gameId' 
04:30:34   'DataFrame' object has no attribute 'gameId' 
04:30:47   'DataFrame' object has no attribute 'gameId' 
04:31:07   'DataFrame' object has no attribute 'gameId' 
04:31:55   39400: 4544611284
04:35:00   'DataFrame' object has no attribute 'gameId' 
04:35:29   39474: 4541727838 'summonerId'
04:35:57   'DataFrame' object has no attribute 'gameId' 
04:36:03   39500: 4511843470
04:38:14   'DataFrame' object has no attribute 'gameId' 
04:38:46   'DataFrame' object has no attribute 'gameId' 
04:39:09   'DataFrame' object has no attribute 'gameId' 
04:40:20   39600: 4458104153
04:43:43   39664: 4502668773 'summonerId'
04:44:26   39700: 4469900853


In [111]:
print(j)

39171


In [109]:
MATCHES.tail()

,gameCreation,gameDuration,gameId,gameMode,gameType,gameVersion,mapId,platformId,queueId,seasonId,...,team2_4_lane,team2_4_role,team2_5_championId,team2_5_spell1Id,team2_5_spell2Id,team2_5_accountId,team2_5_summonerName,team2_5_summonerId,team2_5_lane,team2_5_role
38966,1593509795770,2137,4479861189,CLASSIC,MATCHED_GAME,10.13.326.4870,11,KR,420,13,...,BOTTOM,DUO_SUPPORT,245,4,11,AqraIK5579cEijNovrM1qSIRAnhfXi8idismhRMSVwftEq...,dimemaythgbedha,FIZG8K2jYKJXpWws-etWKeqE6MQ7byQ7ZHOwK35YC8c1xQU,JUNGLE,NONE
38967,1594374792223,1082,4499784140,CLASSIC,MATCHED_GAME,10.14.327.8505,11,KR,420,13,...,NONE,DUO_SUPPORT,106,11,4,iUciSBA_ccas_6QrCTg9MOT3qPUHfzAdpfwkYVmOcJvQwA...,Keep calm and C,mGc_NpEOPTEtfJtzWx0oot7yBigONZfX4mBMmvPAEJyv4mc,NONE,DUO_SUPPORT
38968,1590937805122,2023,4420092364,CLASSIC,MATCHED_GAME,10.11.322.2991,11,KR,430,13,...,BOTTOM,DUO_CARRY,39,14,4,IuougcPWMMp4kWlzIWAdJOgmSLrwAYdjTJqMmS3b4mE8je...,라넌큘러스,dygPkJ-IRrUGHSzQoNXqUzoxyBrQ-FJWqF2UKj8E5uCSGic,MIDDLE,SOLO
38969,1592999354990,1754,4467802580,CLASSIC,MATCHED_GAME,10.13.325.7485,11,KR,420,13,...,BOTTOM,DUO_SUPPORT,266,4,12,gF8YqddwaqXsn_zkzzjD1MN3bIRPlp4uZVvmDpUGdXB9NZM,운도리,88maOsuXTEqdGgTmF-9NrwLM35LWfWaXiKN73mItbKntAQ0,TOP,SOLO
38970,1595745787447,1181,4532290004,NEXUSBLITZ,MATCHED_GAME,10.15.328.8759,21,KR,1300,13,...,NONE,DUO_SUPPORT,67,4,21,30JPMzxxQTMg993W0QrCKrTtiPXB33ztHUHd7plu8QXa,꿈꾸는 도토리,8Wt61XuiQ_df-XuOVNQVy2tM_dlj8fKkwzU9rDgQqs2zoQ,NONE,DUO


# 매치 데이터 정리

* gameId
* seasonId
* gameVersion
* participants
    - championId
    - spell1Id
    - spell2Id
* participantIdentities
    - accountId
    - summonerName
    - summonerId

In [110]:
match_df = json_normalize(match_req.json())
match_df.drop(columns=['gameCreation', 'gameMode', 'gameType',
                       'mapId', 'queueId'], inplace=True)
# match_df.set_index('gameId', inplace=True)
match_df.head()

,gameDuration,gameId,gameVersion,participantIdentities,participants,platformId,seasonId,teams
0,1867,4530461969,10.15.328.8759,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,13,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T..."


In [119]:
match_df = json_normalize(match_req.json())
match_df.drop(columns=['gameCreation', 'gameMode', 'gameType',
                       'mapId', 'queueId'], inplace=True)

prtcp_key_list = ['championId', 'spell1Id', 'spell2Id']
prtcpId_key_list = ['accountId', 'summonerName', 'summonerId']

team1 = match_df['teams'].values[0][0]['win']
team2 = match_df['teams'].values[0][1]['win']

if team1 == 'Win':
    match_df['win'] = 'team1'
else:
    match_df['win'] = 'team2'
    
def select_lane(lane, role):
    if lane == 'TOP':
        return 'TOP'
    elif lane == 'JUNGLE':
        return 'JUG'
    elif lane == 'MIDDLE':
        return 'MID'
    elif lane == 'BOTTOM' and role == 'DUO_CARRY':
        return 'ADC'
    elif lane == 'BOTTOM' and role == 'DUO_SUPPORT':
        return 'SUP'
    else:
        return None
    
for i in range(5):
    lane = match_df['participants'].values[0][i]['timeline']['lane']
    role = match_df['participants'].values[0][i]['timeline']['role']
    
    lane = select_lane(lane, role)
    
    for key in prtcp_key_list:
        match_df['team1_{0}_{1}'.format(lane, key)] = match_df['participants'].values[0][i][key]
        
    for key in prtcpId_key_list:
        match_df['team1_{0}_{1}'.format(lane, key)] = match_df['participantIdentities'].values[0][i]['player'][key]
        
for i in range(5,10):
    lane = match_df['participants'].values[0][i]['timeline']['lane']
    role = match_df['participants'].values[0][i]['timeline']['role']
    
    lane = select_lane(lane, role)
    
    for key in prtcp_key_list:
        match_df['team2_{0}_{1}'.format(lane, key)] = match_df['participants'].values[0][i][key]
        
    for key in prtcpId_key_list:
        match_df['team2_{0}_{1}'.format(lane, key)] = match_df['participantIdentities'].values[0][i]['player'][key]
        
    
match_df.drop(columns=['teams', 'participants', 'participantIdentities'], inplace=True)
match_df.set_index('gameId', inplace=True)
match_df.head()

,gameDuration,gameVersion,platformId,seasonId,win,team1_TOP_championId,team1_TOP_spell1Id,team1_TOP_spell2Id,team1_TOP_accountId,team1_TOP_summonerName,...,team2_ADC_spell2Id,team2_ADC_accountId,team2_ADC_summonerName,team2_ADC_summonerId,team2_SUP_championId,team2_SUP_spell1Id,team2_SUP_spell2Id,team2_SUP_accountId,team2_SUP_summonerName,team2_SUP_summonerId
gameId,,,,,,,,,,,,,,,,,,,,,
4530461969,1867,10.15.328.8759,KR,13,team1,68,12,4,B0YBK1wCNNubEk-tRp7qmN601ubjdWLO6C7bkRFH50clSkA,사 용 수,...,7,1obHO2Y4DAZlobK7396eUH3-L85RjjkWHANvsohiOaVti2...,wohennangu0,zIHvAi4E8ZKpdMKsyY4nqFZ6tO4KesA7go4W6Nu2FeClxwQ,875,4,14,04bIyaoAJLviZ-6-Bofb7zYVWeU6Tnk5uAz-43CMLdE94G...,실망자,cdya4bpOdk15ISCADNsLWs0GolBDkHJ5-zgxU-wsJz0zCY0


In [130]:
match_train_df = match_df.drop(columns=['win', 'gameDuration', 'gameVersion', 'platformId', 'seasonId'])

orders = []

for team in ['team1', 'team2']:
    for lane in ['TOP', 'JUG', 'MID', 'ADC', 'SUP']:
        match_train_df.drop(columns=[team+'_'+lane+'_accountId', 
                                     team+'_'+lane+'_summonerName', 
                                     team+'_'+lane+'_summonerId'], 
                            inplace=True)
        
        orders.append(team+'_'+lane+'_championId')
        orders.append(team+'_'+lane+'_spell1Id')
        orders.append(team+'_'+lane+'_spell2Id')
        
match_train_df = match_train_df[orders]

match_train_X_df = pd.get_dummies(match_train_df, columns=match_train_df.columns)
match_train_y_df = match_df['win']

match_train_df.head()

,team1_TOP_championId,team1_TOP_spell1Id,team1_TOP_spell2Id,team1_JUG_championId,team1_JUG_spell1Id,team1_JUG_spell2Id,team1_MID_championId,team1_MID_spell1Id,team1_MID_spell2Id,team1_ADC_championId,...,team2_JUG_spell2Id,team2_MID_championId,team2_MID_spell1Id,team2_MID_spell2Id,team2_ADC_championId,team2_ADC_spell1Id,team2_ADC_spell2Id,team2_SUP_championId,team2_SUP_spell1Id,team2_SUP_spell2Id
gameId,,,,,,,,,,,,,,,,,,,,,
4530461969,68,12,4,106,4,11,61,1,4,22,...,4,142,12,4,235,4,7,875,4,14


In [131]:
match_train_X_df.head()

,team1_TOP_championId_68,team1_TOP_spell1Id_12,team1_TOP_spell2Id_4,team1_JUG_championId_106,team1_JUG_spell1Id_4,team1_JUG_spell2Id_11,team1_MID_championId_61,team1_MID_spell1Id_1,team1_MID_spell2Id_4,team1_ADC_championId_22,...,team2_JUG_spell2Id_4,team2_MID_championId_142,team2_MID_spell1Id_12,team2_MID_spell2Id_4,team2_ADC_championId_235,team2_ADC_spell1Id_4,team2_ADC_spell2Id_7,team2_SUP_championId_875,team2_SUP_spell1Id_4,team2_SUP_spell2Id_14
gameId,,,,,,,,,,,,,,,,,,,,,
4530461969,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [128]:
match_train_y_df.head()

gameId
4530461969    team1
Name: win, dtype: object

In [ ]:
match_df['team2_TOP_summonerName']

In [89]:
match_df['participantIdentities'].values[0][0]

{'participantId': 1,
 'player': {'platformId': 'KR',
  'accountId': 'B0YBK1wCNNubEk-tRp7qmN601ubjdWLO6C7bkRFH50clSkA',
  'summonerName': '사 용 수',
  'summonerId': 'XoeObrc8p2RKmU1fglPJXxMrJyhQjs2kupP9_jiMlcZ1X4E',
  'currentPlatformId': 'KR',
  'currentAccountId': 'B0YBK1wCNNubEk-tRp7qmN601ubjdWLO6C7bkRFH50clSkA',
  'matchHistoryUri': '/v1/stats/player_history/KR/204196076',
  'profileIcon': 3872}}